<a href="https://colab.research.google.com/github/JoshiBabuG/Heroku-Deployment-master/blob/master/Linear_Regression_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [2]:
#importing important libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#loading the dataset to pandas dataframe
url = 'https://raw.githubusercontent.com/Afsaan/Linear-Regression-with-TensorFlow-2.x/master/graduate_admission.csv'
df = pd.read_csv(url)

#printing top 5 rows
df.head()

,index,gre,toefl,uni_rating,sop,lor,cgpa,research,admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [5]:
# Basic information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       400 non-null    int64  
 1   gre         400 non-null    int64  
 2   toefl       400 non-null    int64  
 3   uni_rating  400 non-null    int64  
 4   sop         400 non-null    float64
 5   lor         400 non-null    float64
 6   cgpa        400 non-null    float64
 7   research    400 non-null    int64  
 8   admit       400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [6]:
#target 
target = 'admit'

# Categorical Columns
categorical_columns = ['research']
df = pd.get_dummies(df,columns = categorical_columns)

In [7]:
df

,index,gre,toefl,uni_rating,sop,lor,cgpa,admit,research_0,research_1
0,1,337,118,4,4.5,4.5,9.65,0.92,0,1
1,2,324,107,4,4.0,4.5,8.87,0.76,0,1
2,3,316,104,3,3.0,3.5,8.00,0.72,0,1
3,4,322,110,3,3.5,2.5,8.67,0.80,0,1
4,5,314,103,2,2.0,3.0,8.21,0.65,1,0
...,...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,0.82,0,1
396,397,325,107,3,3.0,3.5,9.11,0.84,0,1
397,398,330,116,4,5.0,4.5,9.45,0.91,0,1
398,399,312,103,3,3.5,4.0,8.78,0.67,1,0


In [8]:
# Seperating data into Training and Validation Data
training_df = df.sample(frac=0.85, random_state=101)
validation_df = df.drop(training_df.index)

In [9]:
print(f'[INFO] shape of training data {training_df.shape}')
print(f'[INFO] shape of validation data {validation_df.shape}')

[INFO] shape of training data (340, 10)
[INFO] shape of validation data (60, 10)


In [10]:
# Removing labels from data and converting to numpy array
training_labels = training_df.pop(target).values
validation_labels = validation_df.pop(target).values

In [13]:
# basic stats of the training dataset
stats = training_df.describe().transpose()
print(stats)

            count        mean         std  ...      50%     75%     max
index       340.0  200.150000  114.640690  ...  199.500  297.25  400.00
gre         340.0  316.561765   11.390585  ...  316.500  325.00  340.00
toefl       340.0  107.247059    6.120050  ...  107.000  111.25  120.00
uni_rating  340.0    3.070588    1.134477  ...    3.000    4.00    5.00
sop         340.0    3.388235    1.017912  ...    3.500    4.00    5.00
lor         340.0    3.451471    0.897307  ...    3.500    4.00    5.00
cgpa        340.0    8.584794    0.605356  ...    8.565    9.08    9.92
research_0  340.0    0.476471    0.500182  ...    0.000    1.00    1.00
research_1  340.0    0.523529    0.500182  ...    1.000    1.00    1.00

[9 rows x 8 columns]


In [14]:
# scaling data w.r.t training_df
normalization_training_df = (training_df-stats['mean'])/stats['std']
normalization_validation_df = (validation_df-stats['mean'])/stats['std']

In [15]:
# converting the data to numpy array
training_data = normalization_training_df.to_numpy()
validation_data = normalization_validation_df.to_numpy()
# data type
print(f'[INFO] type of training data {type(training_data)}')
print(f'[INFO] type of validation data {type(validation_data)}')

[INFO] type of training data <class 'numpy.ndarray'>
[INFO] type of validation data <class 'numpy.ndarray'>


In [16]:
# prepare tf.data.Dataset for training
def prepare_dataset(data , labels , batch, shuffle):
  dataset = tf.data.Dataset.from_tensor_slices((data , labels))
  dataset = dataset.shuffle(shuffle)
  dataset = dataset.batch(batch).prefetch(1)
  return dataset



In [17]:
# preparing Dataset for training
batch_size = 32
buffer = 100
training_dataset = prepare_dataset(training_data, training_labels, batch_size, buffer)
validation_dataset = prepare_dataset(validation_data, validation_labels, batch_size, buffer)

In [19]:
print(type(training_data))

<class 'numpy.ndarray'>


In [20]:
import tensorflow as tf

In [21]:
# creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6 , activation='relu'), #first hidden layer
        tf.keras.layers.Dense(3 , activation='sigmoid'),# second hiddeen layer
        tf.keras.layers.Dense(7 , activation='relu'),

        tf.keras.layers.Dense(1)
        ])